# Демонстрация использования классов `NormalizationCoefficientGenerator` и `OscillogramNormalizer`

Этот ноутбук показывает процесс работы с модулем нормализации:
1.  **`NormalizationCoefficientGenerator`** (`normalization.normalization`): Используется для анализа сырых осциллограмм и генерации CSV-файла с коэффициентами нормализации (`norm.csv`). Этот файл содержит информацию о предполагаемых номинальных значениях, типе подключения (первичка/вторичка) и другие параметры для каждого сигнала в каждой осциллограмме.
2.  **`OscillogramNormalizer`** (`normalization.normalization`): Использует сгенерированный `norm.csv` для приведения данных осциллограмм к нормализованному виду (обычно к относительным единицам).

Также будут продемонстрированы статические методы `NormalizationCoefficientGenerator` для управления файлами коэффициентов.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/comtrade_files/for_normalization_1/`. Для демонстрации эти файлы копируются во временную рабочую директорию (`temp_normalization_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import csv # For creating additions_csv_path
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from normalization.normalization import NormalizationCoefficientGenerator, OscillogramNormalizer

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_for_norm_path = os.path.join(base_sample_data_dir, "comtrade_files", "for_normalization_1")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_norm = "temp_normalization_notebook_data"
# Директория, куда будут скопированы осциллограммы для генерации коэффициентов
notebook_source_dir_norm_coeffs = os.path.join(notebook_base_temp_dir_norm, "source_for_coeff_gen")
# Директория для других выходных файлов этого ноутбука (слияния и т.д.)
notebook_output_files_dir = os.path.join(notebook_base_temp_dir_norm, "output_files")

if os.path.exists(notebook_base_temp_dir_norm):
    shutil.rmtree(notebook_base_temp_dir_norm)
os.makedirs(notebook_source_dir_norm_coeffs)
os.makedirs(notebook_output_files_dir)

# --- Копирование необходимых файлов для демонстрации ---
# Файлы из tests/sample_data/comtrade_files/for_normalization_1/
# (osc_norm_voltage.cfg, osc_noise_current.cfg, osc_high_primary.cfg, osc_distorted.cfg и их .dat)
files_to_copy = [
    "osc_norm_voltage.cfg", "osc_norm_voltage.dat",
    "osc_noise_current.cfg", "osc_noise_current.dat",
    "osc_high_primary.cfg", "osc_high_primary.dat",
    "osc_distorted.cfg", "osc_distorted.dat"
]
all_files_copied = True
for file_name in files_to_copy:
    src_path = os.path.join(comtrade_samples_for_norm_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, notebook_source_dir_norm_coeffs)
    else:
        print(f"ОШИБКА: Исходный файл для теста не найден: {src_path}")
        all_files_copied = False
    
if not all_files_copied:
    print("Не все файлы были скопированы. Демонстрация может быть неполной.")
else:
    print(f"Демонстрационные файлы для генерации коэффициентов скопированы в: {notebook_source_dir_norm_coeffs}")
    print("Содержимое:")
    for item in os.listdir(notebook_source_dir_norm_coeffs): print(f"- {item}")

## Часть 1: `NormalizationCoefficientGenerator` - Генерация коэффициентов нормализации

In [ ]:
generated_norm_csv_path = os.path.join(notebook_source_dir_norm_coeffs, "norm.csv")

coeff_generator = NormalizationCoefficientGenerator(
    osc_path=notebook_source_dir_norm_coeffs, # Директория, где лежат CFG для анализа и куда будет сохранен norm.csv
    prev_norm_csv_path="", 
    bus=1 
)

print("\nЗапуск генерации коэффициентов нормализации...")
# В оригинальном коде NormalizationCoefficientGenerator.normalization() не принимает is_print_message.
# tqdm используется внутри для прогресс-бара.
coeff_generator.normalization() # Если класс был обновлен для is_print_message/verbose_logging, это можно добавить

if os.path.exists(generated_norm_csv_path):
    print(f"\nФайл коэффициентов '{generated_norm_csv_path}' успешно создан.")
    norm_coeffs_df = pd.read_csv(generated_norm_csv_path)
    print("Содержимое сгенерированного norm.csv:")
    try:
        from IPython.display import display
        display(norm_coeffs_df)
    except ImportError:
        print(norm_coeffs_df)
else:
    print(f"\nОшибка: Файл '{generated_norm_csv_path}' не был создан.")

### Статические методы `NormalizationCoefficientGenerator` (управление CSV)

In [ ]:
norm_additions_content = [
    ["name", "norm", "1Ub_PS", "1Ub_base", "1Ub_h1", "1Ub_hx"],
    ["osc_new_entry", "YES", "s", "100", "95.0", "5.0"]
]
additions_csv_path = os.path.join(notebook_output_files_dir, "norm_additions.csv")
with open(additions_csv_path, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(norm_additions_content)

merged_norm_path = os.path.join(notebook_output_files_dir, "norm_merged.csv")
print(f"\nОбъединение {generated_norm_csv_path} и {additions_csv_path}...")
if os.path.exists(generated_norm_csv_path):
    NormalizationCoefficientGenerator.merge_normalization_files_static(
        input_paths_or_folder=[generated_norm_csv_path, additions_csv_path],
        output_csv_path=merged_norm_path,
        verbose_logging_method=True # Предполагаем, что стат. методы тоже могли быть обновлены
    )
    if os.path.exists(merged_norm_path):
        print("Содержимое объединенного файла:")
        try: from IPython.display import display; display(pd.read_csv(merged_norm_path))
        except ImportError: print(pd.read_csv(merged_norm_path))
else:
    print(f"Файл {generated_norm_csv_path} не найден, объединение пропущено.")

print("\nДемонстрация correct_df_static (пример вызова, файл для исправления не создается в этом примере):")
corrected_df_path = os.path.join(notebook_output_files_dir, "norm_corrected.csv")
# Для реальной демонстрации, merged_norm_path должен содержать строки с norm='NO' или 'hz'
# NormalizationCoefficientGenerator.correct_df_static(merged_norm_path, corrected_df_path, verbose_logging_method=True)
print("Вызов correct_df_static закомментирован, т.к. требует специально подготовленного файла.")

## Часть 2: `OscillogramNormalizer` - Применение коэффициентов нормализации

In [ ]:
if not os.path.exists(generated_norm_csv_path):
    print(f"Файл коэффициентов {generated_norm_csv_path} не найден. Демонстрация OscillogramNormalizer невозможна.")
else:
    normalizer = OscillogramNormalizer(norm_coef_file_path=generated_norm_csv_path, is_print_message=True) # is_print_message для конструктора OscillogramNormalizer

    osc_to_normalize_path = os.path.join(notebook_source_dir_norm_coeffs, "osc_norm_voltage.cfg")

    try:
        osc_obj_to_norm = Oscillogram(osc_to_normalize_path)
        print(f"\nЗагружена осциллограмма для нормализации: {osc_obj_to_norm.file_hash} ({osc_to_normalize_path})")
        
        original_df = osc_obj_to_norm.data_frame.copy()
        
        print("Нормализация данных...")
        # file_identifier должен соответствовать значению в столбце 'name' в norm.csv
        # NormalizationCoefficientGenerator сохраняет file_hash как 'name'.
        normalized_df = normalizer.normalize_bus_signals(
            bus_df=original_df, 
            file_identifier=osc_obj_to_norm.file_hash, # Используем хеш, как и при генерации
            is_print_error=True
        )

        if normalized_df is not None:
            print("\nСравнение данных до и после нормализации (канал 'U_Normal_100V'):")
            # Убедимся, что столбец существует перед доступом
            if 'U_Normal_100V' in original_df.columns and 'U_Normal_100V' in normalized_df.columns:
                comparison_df = pd.DataFrame({
                    'Original U_Normal_100V': original_df['U_Normal_100V'].head(),
                    'Normalized U_Normal_100V': normalized_df['U_Normal_100V'].head()
                })
                try: from IPython.display import display; display(comparison_df)
                except ImportError: print(comparison_df)
                print("Ожидается, что нормализованные значения будут около 0.333, если исходный сигнал ~100В, а _base=100.")
                print("(Это связано с тем, что нормализация напряжений в OscillogramNormalizer делит на 3 * _base)")
            else:
                print("Столбец 'U_Normal_100V' не найден в одном из DataFrame.")
        else:
            print("Нормализация не была применена или вернула None.")

    except Exception as e:
        print(f"Ошибка при демонстрации OscillogramNormalizer: {e}")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_norm):
        shutil.rmtree(notebook_base_temp_dir_norm)
        print(f"\nВременная директория {notebook_base_temp_dir_norm} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_norm} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_norm}: {e}")